In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.cross_validation import  train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
np.random.seed(7)

#load the dataset
stock_dataset = pd.read_csv('sp500_joined_closing_prices.csv')
stock_dataset.head()

,Date,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,...,WYNN,XEL,XRX,XLNX,XL,XYL,YUM,ZBH,ZION,ZTS
0,2009-12-31,66.870346,18.608963,NaN,8.73,34.275032,10.102340,36.779999,9.68,39.608784,...,43.206295,15.368580,NaN,20.342655,15.397597,NaN,19.994463,55.736500,12.181588,NaN
1,2010-01-04,67.153427,18.770958,NaN,8.74,34.745800,10.275106,37.090000,9.70,39.510929,...,47.457897,15.267188,NaN,20.602419,15.725214,NaN,20.063066,56.594555,12.656321,NaN
2,2010-01-05,66.732811,18.619303,NaN,8.53,34.960529,10.293294,37.700001,9.71,39.276093,...,50.344254,15.086126,NaN,20.342655,15.616008,NaN,19.994463,58.386120,13.102568,NaN
3,2010-01-06,67.679230,18.722702,NaN,8.40,35.332191,10.238736,37.619999,9.57,39.618572,...,49.683880,15.115088,NaN,20.204662,15.456399,NaN,19.851519,58.367264,14.241924,NaN
4,2010-01-07,67.727753,18.877800,NaN,8.40,35.299152,9.993223,36.889999,9.47,39.608784,...,50.744930,15.049910,NaN,20.001724,15.456399,NaN,19.845798,59.706226,15.837014,NaN


In [3]:
stock_dataset.dtypes

Date      object
MMM      float64
ABT      float64
ABBV     float64
ABMD     float64
ACN      float64
ATVI     float64
ADBE     float64
AMD      float64
AAP      float64
AES      float64
AET      float64
AMG      float64
AFL      float64
A        float64
APD      float64
AKAM     float64
ALK      float64
ALB      float64
ARE      float64
ALXN     float64
ALGN     float64
ALLE     float64
AGN      float64
ADS      float64
LNT      float64
ALL      float64
GOOGL    float64
GOOG     float64
MO       float64
          ...   
VRTX     float64
VIAB     float64
V        float64
VNO      float64
VMC      float64
WMT      float64
WBA      float64
DIS      float64
WM       float64
WAT      float64
WEC      float64
WFC      float64
WELL     float64
WDC      float64
WU       float64
WRK      float64
WY       float64
WHR      float64
WMB      float64
WLTW     float64
WYNN     float64
XEL      float64
XRX      float64
XLNX     float64
XL       float64
XYL      float64
YUM      float64
ZBH      float

In [9]:
stock_dataset['Date'] = pd.to_datetime(stock_dataset['Date'])

stock_dataset.set_index('Date',inplace=True)
stock_dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2117 entries, 2009-12-31 to 2018-05-30
Columns: 505 entries, MMM to ZTS
dtypes: float64(505)
memory usage: 8.2 MB


In [10]:
stock_dataset.sort_index(inplace=True)

In [40]:
#normalize data
scaler = MinMaxScaler(feature_range=(0,1))

stock_dataset = stock_dataset.values
stock_dataset

array([[ 66.870346,  18.608963,   0.      , ...,  55.7365  ,  12.181588,
          0.      ],
       [ 67.153427,  18.770958,   0.      , ...,  56.594555,  12.656321,
          0.      ],
       [ 66.732811,  18.619303,   0.      , ...,  58.38612 ,  13.102568,
          0.      ],
       ...,
       [199.029999,  62.369999, 101.080002, ..., 111.080002,  57.049999,
         83.669998],
       [195.740005,  61.299999,  99.470001, ..., 109.269997,  54.919998,
         83.150002],
       [198.679993,  62.07    , 103.010002, ..., 110.650002,  55.529999,
         84.449997]])

In [42]:
stock_close = scaler.fit_transform(stock_dataset)
stock_close

array([[0.03541255, 0.04842943, 0.        , ..., 0.13083924, 0.        ,
        0.        ],
       [0.03686115, 0.05187748, 0.        , ..., 0.14064667, 0.01024049,
        0.        ],
       [0.03470875, 0.04864951, 0.        , ..., 0.16112398, 0.01986651,
        0.        ],
       ...,
       [0.71170733, 0.97988079, 0.82888569, ..., 0.76340681, 0.96785908,
        0.97467495],
       [0.69487159, 0.95710589, 0.81568321, ..., 0.74271874, 0.92191271,
        0.96861749],
       [0.70991626, 0.97349532, 0.84471225, ..., 0.75849198, 0.93507107,
        0.98376118]])

In [20]:
#split data into train and test
train_size = int(len(stock_close)* 0.7)
test_size = len(stock_close) - train_size

stock_train, stock_test = stock_close[0:train_size, :], stock_close[train_size:len(stock_close), :]

print('Split data into train and test: ', len(stock_train), len(stock_test))

Split data into train and test:  1481 636


In [21]:
#need to now convert the data into time series looking back over a period of days...e.g. use last 7 days to predict price

def create_ts(ds, series):
    X, Y =[], []
    for i in range(len(ds)-series - 1):
        item = ds[i:(i+series), 0]
        X.append(item)
        Y.append(ds[i+series, 0])
    return np.array(X), np.array(Y)

series = 7

trainX, trainY = create_ts(stock_train, series)
testX, testY = create_ts(stock_test, series)


In [22]:
trainX

array([[0.03541255, 0.03686115, 0.03470875, ..., 0.03980012, 0.04224225,
        0.04083507],
       [0.03686115, 0.03470875, 0.03955182, ..., 0.04224225, 0.04083507,
        0.04112465],
       [0.03470875, 0.03955182, 0.03980012, ..., 0.04083507, 0.04112465,
        0.04000708],
       ...,
       [0.45150014, 0.45226441, 0.45264671, ..., 0.44538615, 0.44595928,
        0.4529332 ],
       [0.45226441, 0.45264671, 0.4539361 , ..., 0.44595928, 0.4529332 ,
        0.43979759],
       [0.45264671, 0.4539361 , 0.44538615, ..., 0.4529332 , 0.43979759,
        0.43669269]])

In [23]:
trainX[0]

array([0.03541255, 0.03686115, 0.03470875, 0.03955182, 0.03980012,
       0.04224225, 0.04083507])

In [39]:
#reshape into  LSTM format - samples, steps, features
# trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 503))
# testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 503))

trainX.shape

(1473, 7, 1)

In [37]:
#build the model
model = Sequential()
model.add(LSTM(4, input_shape=(series, 503)))
model.add(Dense(503))
model.compile(loss='mse', optimizer='adam')
#fit the model
model.fit(trainX, trainY, epochs=500, batch_size=32)

ValueError: Error when checking input: expected lstm_3_input to have shape (7, 503) but got array with shape (7, 1)

In [29]:
#test this model out
trainPredictions = model.predict(trainX)
testPredictions = model.predict(testX)
#unscale predictions
trainPredictions = scaler.inverse_transform(trainPredictions)
testPredictions = scaler.inverse_transform(testPredictions)
trainY = scaler.inverse_transform([trainY])
testY = scaler.inverse_transform([testY])

ValueError: non-broadcastable output operand with shape (1473,1) doesn't match the broadcast shape (1473,505)

In [28]:
#lets calculate the root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredictions[:, 0]))
testScore = math.sqrt(mean_squared_error(testY[0], testPredictions[:, 0]))
print('Train score: %.2f rmse', trainScore)
print('Test score: %.2f rmse', testScore)

TypeError: Singleton array 0.041124648618500514 cannot be considered a valid collection.